In [4]:
import math
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F

In [2]:
def spatial_pyramid_pool(self,previous_conv,num_sample,previous_conv_size,out_pool_size):
    """
    :param previous_conv: a Tensor Vector of Previous Convolution Layer
    :param num_sample: an Int Number of Image in the batch
    :param previous_conv_size: an Int Vector [height,width] of the matrix features size of previous convolution layer
    :param out_pool_size: a Int Vector of Expected output size of max pooling layer

    :return: a tensor vector with shape [1xn] is the concentration of multi -level pooling
    """

    for i in range(len(out_pool_size)) :  # Paper : 6,3,2,1
        h_wid = int(math.ceil(previous_conv_size[0] / out_pool_size[i]))
        w_wid = int(math.ceil(previous_conv_size[1]/ out_pool_size[i]))
        h_pad = (h_wid * out_pool_size[i] - previous_conv_size[0] + 1) /2
        w_pad = (w_wid * out_pool_size[i] - previous_conv_size[0] + 1) /2
        maxpool = nn.Maxpool2d((h_wid,w_wid),stride =(h_wid,w_wid),padding=(h_pad,w_pad))

        x = maxpool(previous_conv)

        if i == 0 :
            spp = x.view(num_sample,-1)
        else :
            spp = torch.cat((spp,x.view(num_sample)),1)
    return spp

In [ ]:
class SPP_NET(nn.Module):
    def __init__(self,opt,input_nc,ndf=64):
        """
        CNN model which adds spp layer so that we can input multi-size tensor
        """
        super(SPP_NET,self).__init__()
        self.output_num = [4,2,1]

        self.conv1 = nn.Conv2d(input_nc,ndf,4,2,1,bias =False)

        self.conv2 = nn.Conv2d(ndf,ndf*2,4,1,1,bias=False)
        self.BN1 = nn.BatchNorm2d(ndf *2)

        self.conv3 = nn.Conv2d(ndf*2,ndf*4,4,1,1,bias=False)
        self.BN2 = nn.BatchNorm2d(ndf*4)

        self.conv4 = nn.Conv2d(ndf*4,ndf *8,4,1,1,bias=False)
        self.BN3 = nn.BatchNorm2d(ndf*8)

        self.conv5 = nn.Conv2d(ndf*8,64,4,1,0,bias=False)
        self.fc1 = nn.Linear(10752,4096)
        self.fc2 = nn.Linear(4096,1000)


    def forward(self,x) :
        x = self.conv1(x)
        x = self.LReLU1(x)

        x = self.conv2(x)
        x = F.leaky_relu(self.BN1(x))

        x = self.conv3(x)
        x = F.leaky_relu(self.BN2(x))

        x = self.conv4(x)
        spp = spatial_pyramid_pool(x,1,[int(x.size(2)),int(x.size(3))],self.output_num)
        fc1 = self.fc1(spp)
        fc2 = self.fc2(fc1)
        s = nn.Sigmoid()
        output = s(fc2)
        return output